In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import pickle

# 配置web驱动器
driver = webdriver.Chrome()
# 设置等待时间
wait = WebDriverWait(driver, 10)

# 打开IMDb网站并执行搜索功能
def search_keyword(keyword):
    driver.get("https://www.imdb.com/")
    input_element = wait.until(EC.presence_of_element_located((By.ID, "suggestion-search")))
    input_element.send_keys(keyword + Keys.ENTER)

# 获得详情页和图片链接
def get_data():
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "ipc-metadata-list-summary-item")))
    xpath_expression = '//a[@class="ipc-metadata-list-summary-item__t"]'
    element = driver.find_element(By.XPATH, xpath_expression)
    element.click()

    soup = BeautifulSoup(driver.page_source, "html.parser")
    target_div = soup.find("div", class_="sc-4e4cc5f9-7 joCxEc")

    if target_div:
        img_element = target_div.find("img")
        title_element = target_div.find("a", class_="ipc-lockup-overlay ipc-focusable")
        
        if img_element and title_element:
            return driver.current_url, img_element["src"], title_element["aria-label"]

    print("未找到目标元素")
    return None, None, None

# 下载图片
def download_image(img_url, path):
    response = requests.get(img_url)
    if response.status_code == 200:
        with open(path, 'wb') as f:
            f.write(response.content)
        print("图片已成功下载：", path)
    else:
        print("下载失败，状态码：", response.status_code)

# 读取和更新本地文件
def update_file(keyword, data):
    with open('movie_url_pic.pickle', 'rb') as f:
        dic = pickle.load(f)
    dic[keyword] = data
    with open('movie_url_pic.pickle', 'wb') as f:
        pickle.dump(dic, f)


if __name__ == '__main__':
    keyword_list =  ['Star Trek: First Contact (1996)',
 'Monty Python and the Holy Grail (1974)',
 'Batman (1989)',
 'Speed (1994)',
 'Trainspotting (1996)',
 'True Lies (1994)',
 'Mission: Impossible (1996)',
 'Die Hard: With a Vengeance (1995)',
 'Get Shorty (1995)',
 'Highlander (1986)']
    for keyword in keyword_list:
        search_keyword(keyword)
        url, img_url, title = get_data()

        if img_url and title:
            print("点击后的网址 URL:", url)
            print("图片链接:", img_url)
            print("电影名称及演员列表:", title)

            pic_dir = f"./pictures/{keyword}.jpg"
            download_image(img_url, pic_dir)
            update_file(keyword, [url, pic_dir])

The chromedriver version (122.0.6261.128) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (123.0.6312.107); currently, chromedriver 123.0.6312.122 is recommended for chrome 123.*, so it is advised to delete the driver in PATH and retry


点击后的网址 URL: https://www.imdb.com/title/tt0117731/?ref_=fn_al_tt_1
图片链接: https://m.media-amazon.com/images/M/MV5BYzMzZmE3MTItODYzYy00YWI5LWFkNWMtZTY5NmU2MDkxYWI1XkEyXkFqcGdeQXVyMjUzOTY1NTc@._V1_QL75_UY281_CR0,0,190,281_.jpg
电影名称及演员列表: View ’Star Trek: First Contact’ Poster
图片已成功下载： ./pictures/Star Trek: First Contact (1996).jpg
未找到目标元素
点击后的网址 URL: https://www.imdb.com/title/tt0096895/?ref_=fn_al_tt_1
图片链接: https://m.media-amazon.com/images/M/MV5BZWQ0OTQ3ODctMmE0MS00ODc2LTg0ZTEtZWIwNTUxOGExZTQ4XkEyXkFqcGdeQXVyNzAwMjU2MTY@._V1_QL75_UX190_CR0,1,190,281_.jpg
电影名称及演员列表: View ’Batman’ Poster
图片已成功下载： ./pictures/Batman (1989).jpg
点击后的网址 URL: https://www.imdb.com/title/tt0111257/?ref_=fn_al_tt_1
图片链接: https://m.media-amazon.com/images/M/MV5BYjc0MjYyN2EtZGRhMy00NzJiLWI2Y2QtYzhiYTU3NzAxNzg4XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_QL75_UX190_CR0,0,190,281_.jpg
电影名称及演员列表: View ’Speed’ Poster
图片已成功下载： ./pictures/Speed (1994).jpg
点击后的网址 URL: https://www.imdb.com/title/tt0117951/?ref_=fn_al_tt_1
图片链接: https:/

In [2]:
# import pickle
# with open('movie_url_pic.pickle', 'rb') as f:
#     dic = pickle.load(f)
